In [1]:
from allensdk.brain_observatory.behavior.behavior_project_cache import BehaviorProjectCache as bpc
from visual_behavior.translator.allensdk_sessions import sdk_utils
import visual_behavior.utilities as vbu
import scipy.misc
import os

from visual_behavior import database as db
import pandas as pd
import numpy as np

from tqdm import tqdm

from multiprocessing import Pool

import shutil

/home/dougo/.local/lib/python3.7/site-packages/xarray/core/merge.py:17: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  PANDAS_TYPES = (pd.Series, pd.DataFrame, pd.Panel)


In [2]:
%widescreen

In [3]:
manifest_path = "/home/dougo/manifest.json"
cache = bpc.from_lims(manifest=manifest_path)

In [11]:
cache.get_experiment_table().sort_values(by='date_of_acquisition',ascending=False).reset_index().head()

,ophys_experiment_id,ophys_session_id,behavior_session_id,container_id,project_code,container_workflow_state,experiment_workflow_state,session_name,session_type,equipment_name,date_of_acquisition,isi_experiment_id,specimen_id,sex,age_in_days,full_genotype,reporter_line,driver_line,imaging_depth,targeted_structure,published_at
0,1003444808,1003313533,1003340085,1000740620,VisualBehaviorTask1B,holding,failed,20200128_492395_1imagesB,OPHYS_1_images_B,CAM2P.4,2020-01-28 21:51:13.000000,973776515,967938399,F,159.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Vip-IRES-Cre],175.0,VISp,NaN
1,1003456269,1003277121,1003302686,999191979,VisualBehaviorTask1B,holding,qc,489056_20200128_5imagesApassive,OPHYS_5_images_A_passive,CAM2P.3,2020-01-28 19:38:04.000000,962029605,951667115,M,177.0,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Sst-IRES-Cre],275.0,VISp,NaN
2,1003442044,1003265930,1003297146,1000740435,VisualBehaviorTask1B,holding,qc,486738_20200128_1imagesBretake,OPHYS_1_images_B,CAM2P.4,2020-01-28 19:08:41.000000,958765629,947329063,F,191.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Vip-IRES-Cre],175.0,VISp,NaN
3,1003456278,1003207357,1003249011,1000740410,VisualBehaviorTask1B,holding,qc,20200128_486737_1ImagesB_retake,OPHYS_1_images_B,CAM2P.3,2020-01-28 16:44:46.000000,959017982,947329076,F,191.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Vip-IRES-Cre],175.0,VISp,NaN
4,1003123177,1002996472,1003033316,1000740620,VisualBehaviorTask1B,holding,failed,20200127_492395_1imagesB,OPHYS_1_images_B,CAM2P.4,2020-01-27 21:03:23.000000,973776515,967938399,F,158.0,Vip-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,[Ai148(TIT2L-GC6f-ICL-tTA2)],[Vip-IRES-Cre],175.0,VISp,NaN


In [50]:
container_id = 1000740435
subset = table.query('container_id == @container_id').sort_values(by='date_of_acquisition',ascending=True).drop_duplicates('ophys_session_id').reset_index()
subset['date_of_acquisition'].min().split(' ')[0]

'2020-01-27'

In [53]:
table = cache.get_experiment_table().sort_values(by='date_of_acquisition',ascending=False).reset_index()
container_ids = table['container_id'].unique()
list_of_dicts = []
for container_id in container_ids:
    temp_dict = {
        'container_id':container_id,
        'container_workflow_state':table.query('container_id == @container_id')['container_workflow_state'].unique()[0],
        'first_acquistion_date':subset['date_of_acquisition'].min().split(' ')[0],
    }
    subset = table.query('container_id == @container_id').sort_values(by='date_of_acquisition',ascending=True).drop_duplicates('ophys_session_id').reset_index()
    
    for idx,row in subset.iterrows():
        temp_dict .update({'session_{}'.format(idx):row['session_type']})
        

    list_of_dicts.append(temp_dict)


In [54]:
pd.DataFrame(list_of_dicts)

,container_id,container_workflow_state,first_acquistion_date,session_0,session_1,session_2,session_3,session_4,session_5,session_6,session_7,session_8,session_9,session_10,session_11,session_12,session_13,session_14,session_15,session_16,session_17,session_18,session_19,session_20,session_21
0,1000740620,holding,2018-08-03,OPHYS_1_images_B,OPHYS_1_images_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,999191979,holding,2020-01-27,OPHYS_1_images_B,OPHYS_2_images_B_passive,OPHYS_3_images_B,OPHYS_4_images_A,OPHYS_5_images_A_passive,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1000740435,holding,2020-01-16,OPHYS_1_images_B,OPHYS_1_images_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000740410,holding,2020-01-27,OPHYS_1_images_B,OPHYS_1_images_B,OPHYS_1_images_B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1002996676,holding,2020-01-24,OPHYS_1_images_A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1002996354,holding,2020-01-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,OPHYS_5_images_B_passive,OPHYS_7_receptive_field_mapping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,989436623,holding,2019-12-03,NaN,NaN,NaN,NaN,NaN,NaN,OPHYS_5_images_H_passive,OPHYS_5_images_H_passive,OPHYS_7_receptive_field_mapping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,992835355,holding,2019-12-03,NaN,NaN,NaN,NaN,NaN,OPHYS_7_receptive_field_mapping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,991913453,holding,2019-12-09,OPHYS_1_images_B,OPHYS_2_images_B_passive,OPHYS_3_images_B,OPHYS_3_images_B,OPHYS_4_images_A,OPHYS_4_images_A,OPHYS_6_images_A,OPHYS_5_images_A_passive,OPHYS_7_receptive_field_mapping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,991913064,holding,2019-12-11,OPHYS_1_images_B,OPHYS_3_images_B,OPHYS_3_images_B,OPHYS_1_images_B,OPHYS_2_images_B_passive,OPHYS_3_images_B,OPHYS_4_images_A,OPHYS_6_images_A,OPHYS_5_images_A_passive,OPHYS_7_receptive_field_mapping,OPHYS_7_receptive_field_mapping,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
